In [1]:
import os
import datetime

import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from keras import backend as K
mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False
%matplotlib inline

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
data_path = "/content/drive/MyDrive/Timeseries_test/data_ts.xlsx"
df = pd.read_excel(data_path,sheet_name = "AllWOs")
df.columns = df.columns.str.lower()
df['actstart'] = df['actstart'].dt.strftime("%d/%m/%Y")
df['actfinish'] = df['actfinish'].dt.strftime("%d/%m/%Y")
df=df.sort_values(by = 'actstart')
df.reset_index(drop = True,inplace = True)

In [4]:
col_usage = ['wonum','routine/non-routine','worktype','estdur','actstart','actfinish',]
df = df[df['bdpocdiscipline'] == 'MECH']
df= df[col_usage]
df['actstart'] = pd.to_datetime(df['actstart'], infer_datetime_format=True)
df['actfinish'] =pd.to_datetime(df['actfinish'], infer_datetime_format=True)
df = df.dropna()

In [5]:
#so cong viec con lai
data_list = {}  
worktype = df['worktype'].unique()
routine = df['routine/non-routine'].unique()

for rou in routine:
  for wtype in worktype:
    try:
      mask = (df['routine/non-routine'] == rou) & (df['worktype'] == wtype)
      CM_non_routine = df[mask]
      CM_non_routine.reset_index(drop = True,inplace = True)
      CM_non_routine = CM_non_routine.dropna()
      date_range = pd.date_range(CM_non_routine['actstart'].min(),CM_non_routine['actfinish'].max())
      df_date_range = pd.DataFrame(data=date_range)
      df_date_range['daily_est'] = 0
      df_date_range=df_date_range.set_index([0])
      for index,row in CM_non_routine.iterrows():
          range_task = pd.date_range(start= row['actstart'], end = row['actfinish'])
          if row['actfinish'] == row['actstart']:
              df_date_range.loc[row['actstart']]+= 0
          else:
              day = (row['actfinish'] - row['actstart']).days
              
              for i in range(len(range_task)):
                    df_date_range.loc[range_task[i]]+= day - i
      name = str(wtype) + '_' + rou
      data_list[name] = df_date_range
    except:
      continue

In [6]:
df_all = pd.concat(data_list.values(), axis =1)
df_all = df_all.fillna(0)

In [7]:
df_all.columns = data_list.keys()

In [8]:
np.sum(df_all['PM_Routine'] < 0)

0

In [9]:
df_all.to_csv('/content/drive/MyDrive/Timeseries_test/HP_preprocessing/not_done_alldata_day.csv')  

In [15]:
df_new = pd.read_csv('/content/drive/MyDrive/Timeseries_test/HP_preprocessing/not_done_alldata_day.csv',index_col = '0')

In [16]:
df_new.columns = df_new.columns.map(lambda x : x + '_DAY_NOT_DONE')

In [18]:
df_new.to_csv('/content/drive/MyDrive/Timeseries_test/HP_preprocessing/not_done_alldata_day.csv')

**day done**

In [5]:
#so cong viec con lai
data_list = {}  
worktype = df['worktype'].unique()
routine = df['routine/non-routine'].unique()

for rou in routine:
  for wtype in worktype:
    try:
      mask = (df['routine/non-routine'] == rou) & (df['worktype'] == wtype)
      CM_non_routine = df[mask]
      CM_non_routine.reset_index(drop = True,inplace = True)
      CM_non_routine = CM_non_routine.dropna()
      date_range = pd.date_range(CM_non_routine['actstart'].min(),CM_non_routine['actfinish'].max())
      df_date_range = pd.DataFrame(data=date_range)
      df_date_range['daily_est'] = 0
      df_date_range=df_date_range.set_index([0])
      for index,row in CM_non_routine.iterrows():
          range_task = pd.date_range(start= row['actstart'], end = row['actfinish'])
          if row['actfinish'] == row['actstart']:
              df_date_range.loc[row['actstart']]+= 0
          else:
              day = (row['actfinish'] - row['actstart']).days
              
              for i in range(len(range_task)):
                    df_date_range.loc[range_task[i]]+= i
      name = str(wtype) + '_' + rou
      data_list[name] = df_date_range
    except:
      continue

In [6]:
df_all = pd.concat(data_list.values(), axis =1)
df_all = df_all.fillna(0)

In [9]:
df_all.columns = data_list.keys()

In [10]:
df_all.columns = df_all.columns.map(lambda x : x + '_DAY_DONE')

In [12]:
df_all

,PM_Routine_DAY_DONE,Routine_Routine_DAY_DONE,General_Routine_DAY_DONE,OVERHAUL_Routine_DAY_DONE,PdM_Routine_DAY_DONE,CFT_Routine_DAY_DONE,CM_Non-Routine_DAY_DONE,REC_Non-Routine_DAY_DONE,MoD_Non-Routine_DAY_DONE
0,,,,,,,,,
2013-12-14,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0
2013-12-15,8,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0
2013-12-16,30,0.0,0.0,0.0,0.0,0.0,2,0.0,0.0
2013-12-17,49,0.0,0.0,0.0,0.0,0.0,3,0.0,0.0
2013-12-18,72,0.0,0.0,0.0,0.0,0.0,4,0.0,0.0
...,...,...,...,...,...,...,...,...,...
2021-12-25,0,347.0,348.0,0.0,0.0,0.0,0,0.0,0.0
2021-12-26,1,348.0,350.0,0.0,0.0,0.0,0,0.0,0.0
2021-12-27,1,350.0,0.0,0.0,0.0,0.0,0,0.0,0.0


In [13]:
df_all.to_csv('/content/drive/MyDrive/Timeseries_test/HP_preprocessing/done_alldata_day.csv')